Mean mean heatmap calculations

In [4]:
import cv2
import numpy as np
import os

In [5]:
def mean_of_mean_heatmaps(image_paths, output_path):
    heatmaps = []
    #reference_shape = None

    for path in image_paths:

        if not os.path.exists(path):
            print(f"Error: File not found: {path}. Skipping.")
            continue

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # já colorido [H, W, 3]

        if img is None:
            print(f"Error: loading image {path} Skipping.")
            continue   

        img = img.astype(np.float32) / 255.0  # normaliza para [0, 1]
        heatmaps.append(img)


    if not heatmaps:
        print("No heatmaps loaded.")
        return
    
    #if len(heatmaps) == 1:
    #    print("Only one heatmap loaded. No mean calculation needed.")
    #    return


    mean_heatmap = np.mean(np.stack(heatmaps), axis=0)  # turns list of 3 heatmaps into 3D array w\ shape [3, 224, 224]
    mean_heatmap -= mean_heatmap.min() # subtracts the minimum value from every pixel so the lowest value is 0
    mean_heatmap /= (mean_heatmap.max() + 1e-8) # divides every pixel by the new maximum value (normalize) ; adds 1e-8 to ensure it wont be divided by 0
    mean_heatmap_uint8 = np.uint8(255 * mean_heatmap)
    colored_heatmap = cv2.applyColorMap(mean_heatmap_uint8, cv2.COLORMAP_JET)


    cv2.imwrite(output_path, colored_heatmap)
    print(f"Mean of mean heatmaps saved at: {output_path}")


In [6]:
def main():

    structures = ["abdomen","femur", "head"] 
    birth_types = ["ces", "vag"]
    datasets = ["train", "test"]
    splits = ["cv1", "cv2","cv3"]
    true = ["correct", "incorrect"]
    confidence = ["high", "low"]

    dfs_by_split = {split: {} for split in splits}  # dict de dicts

    base_input = {
        "cv1": "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/medvit_image_dataset_cv1_confidence_sorted/{structure}_{birth_type}_{dataset}_{true}_{confidence}_confidence",
        "cv2": "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/medvit_image_dataset_cv2_confidence_sorted/{structure}_{birth_type}_{dataset}_{true}_{confidence}_confidence",
        "cv3": "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/medvit_image_dataset_cv3_confidence_sorted/{structure}_{birth_type}_{dataset}_{true}_{confidence}_confidence"
    }
    base_output = "C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/mean_mean_heatmaps/{structure}_{birth_type}_{dataset}_{true}_{confidence}_confidence_mean_heatmap.png"


    for structure in structures:
            for birth_type in birth_types:
                for dataset in datasets:
                    for true_label in true:
                        for confidences in confidence:
                            image_paths = []
                            filename = f"{true_label}_mean_smooth.png"
                            for split in splits:
                                input_folder = base_input[split].format(
                                    structure=structure,
                                    birth_type=birth_type,
                                    dataset=dataset,
                                    true=true_label,
                                    confidence=confidences
                                )

                                heatmap_path = os.path.join(input_folder, filename)
                                if os.path.exists(heatmap_path):
                                    image_paths.append(heatmap_path)
                                else:
                                    print(f"[MISSING] {heatmap_path}")
                            
                            if image_paths:  # Processa com o que existir
                                output_path = base_output.format(
                                    structure=structure,
                                    birth_type=birth_type,
                                    dataset=dataset,
                                    true=true_label,
                                    confidence=confidences
                                )
                                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                                print(f"[PROCESSING] {structure}, {birth_type}, {dataset}, {true_label}, {confidences} -> {len(image_paths)} heatmaps")
                                mean_of_mean_heatmaps(image_paths, output_path)
                            else:
                                print(f"[SKIPPED] No heatmaps found for: {structure}, {birth_type}, {dataset}, {true_label}, {confidences}")

if __name__ == "__main__":
    main()

[MISSING] C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/medvit_image_dataset_cv3_confidence_sorted/abdomen_ces_train_correct_high_confidence\correct_mean_smooth.png
[PROCESSING] abdomen, ces, train, correct, high -> 2 heatmaps
Mean of mean heatmaps saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/mean_mean_heatmaps/abdomen_ces_train_correct_high_confidence_mean_heatmap.png
[MISSING] C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/medvit_image_dataset_cv3_confidence_sorted/abdomen_ces_train_correct_low_confidence\correct_mean_smooth.png
[PROCESSING] abdomen, ces, train, correct, low -> 2 heatmaps
Mean of mean heatmaps saved at: C:/Users/anale/OneDrive/Documentos/Universidade/TESE/RESULTS/Grad_MEDVIT/medvit_image-dataset_confidence_sorted/mean_mean_heatmaps/abdomen_ces_train_correct_low_c